In [23]:
import numpy as np
import os
from methods import *

In [24]:
#user inputs

#load hyperparameters
sizes = ['1_tiny', '2_small', '3_standard', '4_full']
size_folders = ['size_data_f1/' + size for size in sizes]

#datasets
datasets = ['sst2']

#number of output classes
num_classes_list = [2, 2, 2, 6, 2]

#number of augmentations per original sentence
n_aug_list_dict = {'size_data_f1/1_tiny': [32, 32, 32, 32, 32], 
					'size_data_f1/2_small': [32, 32, 32, 32, 32],
					'size_data_f1/3_standard': [16, 16, 16, 16, 4],
					'size_data_f1/4_full': [16, 16, 16, 16, 4]}

#number of words for input
input_size_list = [50, 50, 40, 25, 25]

alphas = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5]

a_methods = ["sr", "ri", "rd", "rs"]

# Number of words for input
input_size_list = [50,50,40,25,25]

#word2vec dictionary
huge_word2vec = 'word2vec/glove.840B.300d.txt'
word2vec_len = 300


In [25]:
def run_rnn(train_file, test_file, num_classes, percent_dataset):

    #initialize model
    model = build_model(input_size, word2vec_len, num_classes)

    #load data
    train_x, train_y = get_x_y(train_file, num_classes, word2vec_len, input_size, word2vec, percent_dataset)
    test_x, test_y = get_x_y(test_file, num_classes, word2vec_len, input_size, word2vec, 1)

    #implement early stopping
    callbacks = [EarlyStopping(monitor='val_loss', patience=3)]

    #train model
    model.fit(	train_x, 
                train_y, 
                epochs=100, 
                callbacks=callbacks,
                validation_split=0.1, 
                batch_size=1024, 
                shuffle=True, 
                verbose=1)
    #model.save('checkpoints/lol')
    #model = load_model('checkpoints/lol')

    #evaluate model
    y_pred = model.predict(test_x)
    test_y_cat = one_hot_to_categorical(test_y)
    y_pred_cat = one_hot_to_categorical(y_pred)
    acc = accuracy_score(test_y_cat, y_pred_cat)

    #clean memory???
    train_x, train_y, test_x, test_y, model = None, None, None, None, None
    gc.collect()

    #return the accuracy
    #print("data with shape:", train_x.shape, train_y.shape, 'train=', train_file, 'test=', test_file, 'with fraction', percent_dataset, 'had acc', acc)
    return acc

In [26]:
def run_cnn(train_file, test_file, num_classes, percent_dataset):

    #initialize model
    model = build_cnn(input_size, word2vec_len, num_classes)

    #load data
    train_x, train_y = get_x_y(train_file, num_classes, word2vec_len, input_size, word2vec, percent_dataset)
    test_x, test_y = get_x_y(test_file, num_classes, word2vec_len, input_size, word2vec, 1)

    #implement early stopping
    callbacks = [EarlyStopping(monitor='val_loss', patience=3)]

    #train model
    model.fit(	train_x, 
                train_y, 
                epochs=100, 
                callbacks=callbacks,
                validation_split=0.1, 
                batch_size=1024, 
                shuffle=True, 
                verbose=1)
    #model.save('checkpoints/lol')
    #model = load_model('checkpoints/lol')

    #evaluate model
    y_pred = model.predict(test_x)
    test_y_cat = one_hot_to_categorical(test_y)
    y_pred_cat = one_hot_to_categorical(y_pred)
    acc = accuracy_score(test_y_cat, y_pred_cat)

    #clean memory???
    train_x, train_y, test_x, test_y, model = None, None, None, None, None
    gc.collect()

    #return the accuracy
    #print("data with shape:", train_x.shape, train_y.shape, 'train=', train_file, 'test=', test_file, 'with fraction', percent_dataset, 'had acc', acc)
    return acc

In [27]:
#for each method
for a_method in a_methods:
    if not os.path.isdir('outputs_f1/'):
        os.mkdir('outputs_f1/')
    

    writer_cnn = open('outputs_f1/cnn_' + a_method + '_' + get_now_str() + '.txt', 'w')
    writer_rnn = open('outputs_f1/rnn_' + a_method + '_' + get_now_str() + '.txt', 'w')


    #for each size dataset
    for size_folder in size_folders:

        writer_cnn.write(size_folder + '\n')
        writer_rnn.write(size_folder + '\n')

        #get all six datasets
        dataset_folders = [size_folder + '/' + s for s in datasets]

        #for storing the performances
        performances_rnn = {alpha:[] for alpha in alphas}
        performances_cnn = {alpha:[] for alpha in alphas}

        #for each dataset
        for i in range(len(dataset_folders)):

            #initialize all the variables
            dataset_folder = dataset_folders[i]
            dataset = datasets[i]
            num_classes = num_classes_list[i]
            input_size = input_size_list[i]
            word2vec_pickle = dataset_folder + '/word2vec.p'
            word2vec = load_pickle(word2vec_pickle)

            #test each alpha value
            for alpha in alphas:

                train_path = dataset_folder + '/train_' + a_method + '_' + str(alpha) + '.txt'
                test_path = 'size_data_f1/test/' + dataset + '/test.txt'
                acc = run_cnn(train_path, test_path, num_classes, percent_dataset=1)
                print("cnn aug", acc)
                
                performances_cnn[alpha].append(acc)
                acc = run_rnn(train_path, test_path, num_classes, percent_dataset=1)
                performances_rnn[alpha].append(acc)
                print("rnn aug", acc)

        writer_cnn.write(str(performances_cnn) + '\n')
        writer_rnn.write(str(performances_rnn)+"\n")
        for alpha in performances_cnn:
            line = str(alpha) + ' : ' + str(sum(performances_cnn[alpha])/len(performances_cnn[alpha]))
            writer_cnn.write(line + '\n')
            print(line)
        for alpha in performances_rnn:
            line = str(alpha) + ' : ' + str(sum(performances_rnn[alpha])/len(performances_rnn[alpha]))
            writer_rnn.write(line + '\n')
            print(line)
        print(performances_cnn)
        print(performances_rnn)

    writer_cnn.close()
    writer_rnn.close()
    
## TODO: add training without augmentation

Epoch 1/100
15/15 [==============================] - 4s 204ms/step - loss: 0.6357 - accuracy: 0.6721 - val_loss: 0.4059 - val_accuracy: 0.9461
Epoch 2/100
15/15 [==============================] - 3s 188ms/step - loss: 0.3294 - accuracy: 0.9460 - val_loss: 0.1182 - val_accuracy: 0.9879
Epoch 3/100
15/15 [==============================] - 3s 194ms/step - loss: 0.0869 - accuracy: 0.9872 - val_loss: 0.0214 - val_accuracy: 0.9994
Epoch 4/100
15/15 [==============================] - 3s 172ms/step - loss: 0.0159 - accuracy: 0.9995 - val_loss: 0.0060 - val_accuracy: 1.0000
Epoch 5/100
15/15 [==============================] - 2s 166ms/step - loss: 0.0050 - accuracy: 1.0000 - val_loss: 0.0029 - val_accuracy: 1.0000
Epoch 6/100
15/15 [==============================] - 2s 167ms/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.0020 - val_accuracy: 1.0000
Epoch 7/100
15/15 [==============================] - 3s 191ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.0015 - val_accuracy: 1.0000

15/15 [==============================] - 2s 165ms/step - loss: 3.3315e-05 - accuracy: 1.0000 - val_loss: 4.8595e-05 - val_accuracy: 1.0000
Epoch 56/100
15/15 [==============================] - 2s 164ms/step - loss: 3.2749e-05 - accuracy: 1.0000 - val_loss: 4.7135e-05 - val_accuracy: 1.0000
Epoch 57/100
15/15 [==============================] - 2s 165ms/step - loss: 3.3355e-05 - accuracy: 1.0000 - val_loss: 4.5770e-05 - val_accuracy: 1.0000
Epoch 58/100
15/15 [==============================] - 2s 164ms/step - loss: 3.0780e-05 - accuracy: 1.0000 - val_loss: 4.4461e-05 - val_accuracy: 1.0000
Epoch 59/100
15/15 [==============================] - 2s 164ms/step - loss: 3.0107e-05 - accuracy: 1.0000 - val_loss: 4.3183e-05 - val_accuracy: 1.0000
Epoch 60/100
15/15 [==============================] - 2s 164ms/step - loss: 2.8113e-05 - accuracy: 1.0000 - val_loss: 4.1940e-05 - val_accuracy: 1.0000
Epoch 61/100
15/15 [==============================] - 2s 164ms/step - loss: 2.9126e-05 - accuracy: 1.

15/15 [==============================] - 12s 816ms/step - loss: 0.0855 - accuracy: 0.9829 - val_loss: 0.0787 - val_accuracy: 0.9818
Epoch 10/100
15/15 [==============================] - 12s 794ms/step - loss: 0.0733 - accuracy: 0.9855 - val_loss: 0.0569 - val_accuracy: 0.9885
Epoch 11/100
15/15 [==============================] - 11s 753ms/step - loss: 0.0605 - accuracy: 0.9883 - val_loss: 0.0479 - val_accuracy: 0.9885
Epoch 12/100
15/15 [==============================] - 11s 762ms/step - loss: 0.0468 - accuracy: 0.9899 - val_loss: 0.0410 - val_accuracy: 0.9891
Epoch 13/100
15/15 [==============================] - 11s 754ms/step - loss: 0.0409 - accuracy: 0.9901 - val_loss: 0.3094 - val_accuracy: 0.9364
Epoch 14/100
15/15 [==============================] - 11s 756ms/step - loss: 0.1423 - accuracy: 0.9661 - val_loss: 0.0374 - val_accuracy: 0.9879
Epoch 15/100
15/15 [==============================] - 11s 756ms/step - loss: 0.0386 - accuracy: 0.9889 - val_loss: 0.0274 - val_accuracy: 0.989

15/15 [==============================] - 2s 158ms/step - loss: 2.5877e-05 - accuracy: 1.0000 - val_loss: 6.2139e-05 - val_accuracy: 1.0000
Epoch 35/100
15/15 [==============================] - 2s 157ms/step - loss: 2.7003e-05 - accuracy: 1.0000 - val_loss: 5.9442e-05 - val_accuracy: 1.0000
Epoch 36/100
15/15 [==============================] - 2s 158ms/step - loss: 2.4527e-05 - accuracy: 1.0000 - val_loss: 5.6604e-05 - val_accuracy: 1.0000
Epoch 37/100
15/15 [==============================] - 2s 158ms/step - loss: 2.3936e-05 - accuracy: 1.0000 - val_loss: 5.4057e-05 - val_accuracy: 1.0000
Epoch 38/100
15/15 [==============================] - 2s 160ms/step - loss: 2.0076e-05 - accuracy: 1.0000 - val_loss: 5.1774e-05 - val_accuracy: 1.0000
Epoch 39/100
15/15 [==============================] - 2s 157ms/step - loss: 1.7978e-05 - accuracy: 1.0000 - val_loss: 4.9948e-05 - val_accuracy: 1.0000
Epoch 40/100
15/15 [==============================] - 2s 156ms/step - loss: 1.8830e-05 - accuracy: 1.

Epoch 88/100
15/15 [==============================] - 2s 159ms/step - loss: 3.8935e-06 - accuracy: 1.0000 - val_loss: 1.4152e-05 - val_accuracy: 1.0000
Epoch 89/100
15/15 [==============================] - 2s 158ms/step - loss: 3.7158e-06 - accuracy: 1.0000 - val_loss: 1.3920e-05 - val_accuracy: 1.0000
Epoch 90/100
15/15 [==============================] - 2s 161ms/step - loss: 3.6377e-06 - accuracy: 1.0000 - val_loss: 1.3697e-05 - val_accuracy: 1.0000
Epoch 91/100
15/15 [==============================] - 2s 160ms/step - loss: 3.2893e-06 - accuracy: 1.0000 - val_loss: 1.3500e-05 - val_accuracy: 1.0000
Epoch 92/100
15/15 [==============================] - 2s 161ms/step - loss: 3.5330e-06 - accuracy: 1.0000 - val_loss: 1.3238e-05 - val_accuracy: 1.0000
Epoch 93/100
15/15 [==============================] - 2s 158ms/step - loss: 3.2546e-06 - accuracy: 1.0000 - val_loss: 1.2984e-05 - val_accuracy: 1.0000
Epoch 94/100
15/15 [==============================] - 2s 161ms/step - loss: 3.4060e-06 -

...

59/59 [==============================] - 13s 201ms/step - loss: 0.5425 - accuracy: 0.7459 - val_loss: 0.1127 - val_accuracy: 0.9803
Epoch 2/100
59/59 [==============================] - 9s 159ms/step - loss: 0.0700 - accuracy: 0.9879 - val_loss: 0.0120 - val_accuracy: 0.9985
Epoch 3/100
59/59 [==============================] - 9s 159ms/step - loss: 0.0082 - accuracy: 0.9989 - val_loss: 0.0035 - val_accuracy: 0.9994
Epoch 4/100
59/59 [==============================] - 9s 159ms/step - loss: 0.0022 - accuracy: 0.9998 - val_loss: 0.0016 - val_accuracy: 0.9997
Epoch 5/100
59/59 [==============================] - 9s 158ms/step - loss: 0.0011 - accuracy: 0.9999 - val_loss: 9.9110e-04 - val_accuracy: 0.9998
Epoch 6/100
59/59 [==============================] - 9s 159ms/step - loss: 6.6515e-04 - accuracy: 0.9998 - val_loss: 7.7872e-04 - val_accuracy: 0.9998
Epoch 7/100
59/59 [==============================] - 9s 159ms/step - loss: 5.1277e-04 - accuracy: 0.9998 - val_loss: 6.4213e-04 - val_accurac

Epoch 22/100
59/59 [==============================] - 43s 721ms/step - loss: 4.3737e-04 - accuracy: 0.9999 - val_loss: 0.0012 - val_accuracy: 0.9998
Epoch 23/100
59/59 [==============================] - 43s 723ms/step - loss: 5.5560e-04 - accuracy: 0.9998 - val_loss: 0.0014 - val_accuracy: 0.9998
rnn aug 0.7861635220125787
Epoch 1/100
59/59 [==============================] - 12s 190ms/step - loss: 0.5182 - accuracy: 0.7728 - val_loss: 0.1070 - val_accuracy: 0.9832
Epoch 2/100
59/59 [==============================] - 10s 162ms/step - loss: 0.0572 - accuracy: 0.9914 - val_loss: 0.0140 - val_accuracy: 0.9982
Epoch 3/100
59/59 [==============================] - 10s 162ms/step - loss: 0.0087 - accuracy: 0.9990 - val_loss: 0.0053 - val_accuracy: 0.9988
Epoch 4/100
59/59 [==============================] - 10s 162ms/step - loss: 0.0030 - accuracy: 0.9997 - val_loss: 0.0024 - val_accuracy: 0.9995
Epoch 5/100
59/59 [==============================] - 10s 161ms/step - loss: 0.0013 - accuracy: 1.00

59/59 [==============================] - 42s 719ms/step - loss: 0.0534 - accuracy: 0.9853 - val_loss: 0.0389 - val_accuracy: 0.9889
Epoch 8/100
59/59 [==============================] - 42s 717ms/step - loss: 0.0402 - accuracy: 0.9886 - val_loss: 0.0980 - val_accuracy: 0.9685
Epoch 9/100
59/59 [==============================] - 42s 720ms/step - loss: 0.0285 - accuracy: 0.9921 - val_loss: 0.0240 - val_accuracy: 0.9935
Epoch 10/100
59/59 [==============================] - 42s 718ms/step - loss: 0.0198 - accuracy: 0.9950 - val_loss: 0.0170 - val_accuracy: 0.9950
Epoch 11/100
59/59 [==============================] - 42s 720ms/step - loss: 0.0172 - accuracy: 0.9952 - val_loss: 0.0150 - val_accuracy: 0.9964
Epoch 12/100
59/59 [==============================] - 42s 717ms/step - loss: 0.0068 - accuracy: 0.9985 - val_loss: 0.0146 - val_accuracy: 0.9964
Epoch 13/100
59/59 [==============================] - 42s 720ms/step - loss: 0.0064 - accuracy: 0.9981 - val_loss: 0.0096 - val_accuracy: 0.9973


Epoch 47/100
59/59 [==============================] - 9s 157ms/step - loss: 1.6568e-04 - accuracy: 0.9999 - val_loss: 0.0017 - val_accuracy: 0.9995
Epoch 48/100
59/59 [==============================] - 9s 157ms/step - loss: 6.7142e-05 - accuracy: 1.0000 - val_loss: 0.0017 - val_accuracy: 0.9995
Epoch 49/100
59/59 [==============================] - 9s 159ms/step - loss: 1.1580e-04 - accuracy: 0.9999 - val_loss: 0.0017 - val_accuracy: 0.9995
Epoch 50/100
59/59 [==============================] - 9s 158ms/step - loss: 1.0257e-04 - accuracy: 0.9999 - val_loss: 0.0017 - val_accuracy: 0.9995
Epoch 51/100
59/59 [==============================] - 9s 157ms/step - loss: 1.5999e-04 - accuracy: 0.9999 - val_loss: 0.0017 - val_accuracy: 0.9995
Epoch 52/100
59/59 [==============================] - 9s 161ms/step - loss: 1.3456e-04 - accuracy: 0.9999 - val_loss: 0.0017 - val_accuracy: 0.9995
Epoch 53/100
59/59 [==============================] - 9s 159ms/step - loss: 1.8096e-04 - accuracy: 0.9999 - val_

KeyboardInterrupt: 

In [ ]:

writer_cnn = open('outputs_f1/cnn_no_aug_' + get_now_str() + '.txt', 'w')
writer_rnn = open('outputs_f1/rnn_no_aug_' + get_now_str() + '.txt', 'w')


#for each size dataset
for size_folder in size_folders:

    writer_cnn.write(size_folder + '\n')
    writer_rnn.write(size_folder + '\n')

    #get all six datasets
    dataset_folders = [size_folder + '/' + s for s in datasets]

    #for storing the performances
    performances_rnn = []
    performances_cnn = []

    #for each dataset
    for i in range(len(dataset_folders)):

        #initialize all the variables
        dataset_folder = dataset_folders[i]
        dataset = datasets[i]
        num_classes = num_classes_list[i]
        input_size = input_size_list[i]
        word2vec_pickle = dataset_folder + '/word2vec.p'
        word2vec = load_pickle(word2vec_pickle)

        train_path = dataset_folder + '/train_orig.txt'
        test_path = 'size_data_f1/test/' + dataset + '/test.txt'
        acc = run_cnn(train_path, test_path, num_classes, percent_dataset=1)
        print("cnn", acc)
        performances_cnn.append(acc)

        acc = run_rnn(train_path, test_path, num_classes, percent_dataset=1)
        performances_rnn.append(acc)
        print("rnn", acc)

    writer_cnn.write(str(performances_cnn) + '\n')
    writer_rnn.write(str(performances_rnn)+"\n")

    print(performances_cnn)
    print(performances_rnn)

writer_cnn.close()
writer_rnn.close()
